### Implementing Data Validation using Protobuf in a Distributed System
**Description**: Use gRPC to implement a distributed system that validates messages using
Protobuf.

**Steps**:
1. Create a .proto file for gRPC service.
2. Implement server-side validation
    - Create a gRPC server
    - Bind the server to an address
    - Start server

In [1]:
# Write your code from here

import grpc
from concurrent import futures
import time
import re
import sys
import threading
from grpc_tools import protoc
import os

# Step 1: Define the .proto schema in memory and write to temp file
PROTO_FILE = "validator.proto"
proto_content = '''
syntax = "proto3";

package validator;

service ValidatorService {
  rpc ValidateUser (UserRequest) returns (ValidationResponse);
}

message UserRequest {
  int32 id = 1;
  string name = 2;
  string email = 3;
}

message ValidationResponse {
  bool is_valid = 1;
  string message = 2;
}
'''

with open(PROTO_FILE, "w") as f:
    f.write(proto_content)

# Step 2: Generate gRPC code from proto
protoc.main((
    '',
    f'-I.',
    f'--python_out=.',
    f'--grpc_python_out=.',
    PROTO_FILE,
))

# Step 3: Import generated code
import validator_pb2
import validator_pb2_grpc

# Step 4: Implement the gRPC server
class ValidatorServicer(validator_pb2_grpc.ValidatorServiceServicer):
    def ValidateUser(self, request, context):
        if not request.name or not request.email:
            return validator_pb2.ValidationResponse(
                is_valid=False,
                message="Name and email are required."
            )
        if not re.match(r"[^@]+@[^@]+\.[^@]+", request.email):
            return validator_pb2.ValidationResponse(
                is_valid=False,
                message="Invalid email format."
            )
        return validator_pb2.ValidationResponse(
            is_valid=True,
            message="User is valid."
        )

def serve():
    server = grpc.server(futures.ThreadPoolExecutor(max_workers=2))
    validator_pb2_grpc.add_ValidatorServiceServicer_to_server(ValidatorServicer(), server)
    server.add_insecure_port('[::]:50051')
    print("🚀 Server started on port 50051")
    server.start()
    server.wait_for_termination()

# Step 5: Implement client to call server
def run_client():
    time.sleep(1)  # wait a bit for server to start
    with grpc.insecure_channel('localhost:50051') as channel:
        stub = validator_pb2_grpc.ValidatorServiceStub(channel)
        user = validator_pb2.UserRequest(id=1, name="Alice", email="alice@example.com")
        response = stub.ValidateUser(user)
        print("✅ Client received:", response.is_valid, "-", response.message)

        # Test with invalid email
        bad_user = validator_pb2.UserRequest(id=2, name="Bob", email="not-an-email")
        response = stub.ValidateUser(bad_user)
        print("❌ Client received:", response.is_valid, "-", response.message)

# Step 6: Run server and client in the same process
if __name__ == "__main__":
    t = threading.Thread(target=serve, daemon=True)
    t.start()
    run_client()

    # Optional cleanup
    os.remove("validator.proto")
    os.remove("validator_pb2.py")
    os.remove("validator_pb2_grpc.py")

ModuleNotFoundError: No module named 'grpc'